In [1]:
import numpy as np
import pandas as pd
import os, sys, shutil, csv

In [2]:
read_cols = 61
name_df = "MW035.tree"
col_names = ["ch{0:02d}".format(i) for i in range(read_cols)]
open_fname = "mainbranch_" + name_df[:-5] + ".csv"

In [3]:
df = pd.read_csv(name_df, names = col_names, delim_whitespace = True, dtype = str)

In [4]:
##Get row number of ch00 == "#tree"
tree_index = list(df.reset_index().query("ch00 == '#tree'").index)
print(len(tree_index))

55000


In [5]:
file_mainbranch = open(open_fname, mode = "a", newline = "")
writer_mainbranch = csv.writer(file_mainbranch)

##Reference all merger trees in this file with for-loop
for t_idx in range(len(tree_index)):
    
    if t_idx != len(tree_index)-1:
        current_df = df[tree_index[t_idx]:tree_index[t_idx+1]].reset_index()
    else:
        current_df = df[tree_index[t_idx]:].reset_index()
    ##Get depth_first_id(ch28) of a last halo in a main branch from the root halo
    last_mainleaf_depthfirst_id = str(current_df.at[current_df.index[1], "ch34"])
    
    ##Go to the last halo in the main branch with depth_first_id(ch28)
    current_halo = int(list(current_df.query("ch28 == @last_mainleaf_depthfirst_id").index)[0])

    writer_mainbranch.writerow(["tree"])
    ##Get the main branch from the last halo with desc_id(ch03) and id(ch01)
    while True:
        current_data = []
        ##Save each parameters
        for col in col_names:
            data = current_df.at[current_df.index[current_halo], col]
            current_data.append(data)
        writer_mainbranch.writerow(current_data)
        
        ##ch03 == -1 indicates root halo in the tree
        if current_df.at[current_df.index[current_halo], "ch03"] == "-1":
            break
        desc_id = str(current_df.at[current_df.index[current_halo], "ch03"])
        ##desc_id indicates an id(ch01) of descendant halo
        current_halo = int(list(current_df.query("ch01 == @desc_id").index)[0])
        if current_halo == 0:
            current_halo += 1

file_mainbranch.close()